# Submitting form data to NACC Flywheel API



In [ ]:
from getpass import getpass
import logging
import os

from flywheel import Client, FileSpec

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

## Flywheel Access

This script requires the API KEY for a user with upload capability for this operation.

To get the API key, login as the user to the NACC Flywheel instance.
Find the "avatar" in the upper right corner (generally a circle with your initials).
Click the avatar dropdown, and select "Profile".
Under "Flywheel Access" at the bottom of the resulting page, click "Generate API Key".

Choose a key name relevant to upload, set the expiration date, and create the API Key.
Copy the API Key since you wont be able to access the value later.
This key should be kept secret.

### Load secret key in script

This script is for the purposes of demonstration, so we just prompt you for the key.

For your own scripting please use appropriate secret management for your execution environment.

In [ ]:
API_KEY = getpass('Enter API_KEY here: ')

## Connect to Flywheel

The API Key is used to connected to Flywheel.
To do this with the Python SDK, we create an SDK client that we will use.

In [ ]:
fw = Client(API_KEY if 'API_KEY' in locals() else os.environ.get('FW_KEY'))

log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

## Identify group for center

Each center is associated with a Flywheel group.
The group label for a center can be found by logging into Flywheel.

**[DETAILS]**

For this tutorial, we use the label `example-adrc`, but you should change this to the label for your center.

In [ ]:
group_label = 'example-adrc'

## Identify project for upload

Each data type has an "ingest" and "sandbox" Flywheel project.
The "ingest" project is for submitting data primary data, and the "sandbox" project is for testing.

For data files that have data for many participants, such as forms, the data is submitted as a CSV where each line is a data record for a participant.
These files are attached to the ingest (or sandbox) project as shown here.

> Data files that have a one to one relationship with participant, such as images, are uploaded differently.

Set `project_prefix` to `sandbox` or `ingest` depending on whether you are uploading test or participant data.
The following code will set the label for the project in the group for your center.

In [ ]:
project_prefix = 'sandbox' # for testing
# project_prefix = 'ingest' # for participant data

project_label = f"{project_prefix}-form"
project_reference = f"{group_label}/{project_label}"

Once the group and ingest project are identified, the SDK client is used to lookup the project.
If the following command fails to find the project, something is wrong with your group and project labels.

In [ ]:
upload_project = fw.lookup(project_reference)
if not upload_project:
    log.error("Failed to find upload project %s", project_reference)

## Upload file

If you have a file on disk, you can upload it directly to the project using code like the following.

In [ ]:
filename = "form-data.csv"
file_path = f"./{filename}"
file_type = 'text/csv'

if upload_project:
    upload_project.upload_file(file_path)
    upload_project.update_file(filename, content_type=file_type)

If on the otherhand, you generate the file contents in memory, create a `flywheel.FileSpec` object that references the contents and then upload the file.

In [ ]:
contents = "your file contents"
file_spec = FileSpec(filename, contents=contents, content_type=file_type)
if upload_project:
    upload_project.upload_file(file_spec)